<a href="https://colab.research.google.com/github/guscldns/TestProject/blob/main/0716/05_learning_rate_scheduling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Schedule Learning Rate for More Improving
Callbacks API 활용  
1) ReduceLROnPlateau  
2) LearningRateScheduler

## Q. 모델의 성능이 더이상 개선되지 않을 때, 왜 Learning Rate을 감소시키는 전략을 쓸까?
이는 얼핏 생각해보면, 반 직관적(counter-intuitive)이지 않은가?  

Learning Rate이 무엇이었는지 다시 한번 상기해보자.  
        
    W ← W - lr * Gradient

### ReduceLROnPlateau
모델이 일정기간(patience)동안 개선되지 않으면(monitor)  
Learning rate을 일정비율(factor)로 감소시킴으로 loss를 낮춤

    tf.keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss",
        factor=0.1, # new_lr = lr * factor.
        patience=10,
        min_delta=0.0001, # threshold for measuring the new optimum, to only focus on significant changes.  
        cooldown=0, # number of epochs to wait before resuming normal operation after lr has been reduced.  
        min_lr=0
    )

In [ ]:
# import tensorflow
import tensorflow as tf

In [ ]:
# dataset download
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

11501568/11490434 [==============================] - 0s 0us/step


In [ ]:
# modeling
def create_model():
    model = tf.keras.models.Sequential([
            tf.keras.layers.Flatten(input_shape=(28, 28)),
            tf.keras.layers.Dense(512, activation='relu'),
            tf.keras.layers.Dropout(0.2),
            tf.keras.layers.Dense(10)
            ])

    return model

In [ ]:
model = create_model()

model.compile(optimizer='adam',
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy'])

In [ ]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=1, min_lr=0)
model.fit(x_train, y_train, epochs=50, validation_split=0.2, callbacks=[reduce_lr])

Epoch 1/50
1500/1500 [==============================] - 8s 5ms/step - loss: 2.3089 - accuracy: 0.0975 - val_loss: 2.3026 - val_accuracy: 0.1064 - lr: 0.0010
Epoch 2/50
1500/1500 [==============================] - 8s 5ms/step - loss: 2.3064 - accuracy: 0.0953 - val_loss: 2.3026 - val_accuracy: 0.1064 - lr: 0.0010
Epoch 3/50
1500/1500 [==============================] - 8s 5ms/step - loss: 2.3108 - accuracy: 0.1009 - val_loss: 2.3026 - val_accuracy: 0.0958 - lr: 2.0000e-04
Epoch 4/50
1500/1500 [==============================] - 8s 6ms/step - loss: 2.3060 - accuracy: 0.0979 - val_loss: 2.3026 - val_accuracy: 0.0958 - lr: 4.0000e-05
Epoch 5/50
1500/1500 [==============================] - 8s 5ms/step - loss: 2.3054 - accuracy: 0.0986 - val_loss: 2.3026 - val_accuracy: 0.0958 - lr: 8.0000e-06
Epoch 6/50
1500/1500 [==============================] - 8s 5ms/step - loss: 2.3062 - accuracy: 0.0989 - val_loss: 2.3026 - val_accuracy: 0.0957 - lr: 1.6000e-06
Epoch 7/50
1500/1500 [====================

### LearningRateScheduler
scheduler함수 설정을 바탕으로, epoch마다 learning rate을 변환

In [ ]:
def scheduler(epoch, lr):
   if epoch < 2:
     return lr
   else:
     return lr * 0.1

In [ ]:
model = create_model()

model.compile(tf.keras.optimizers.Adam(lr=0.001), loss='mse')

model.summary()

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_13 (Flatten)        (None, 784)               0         
                                                                 
 dense_27 (Dense)            (None, 512)               401920    
                                                                 
 dropout_13 (Dropout)        (None, 512)               0         
                                                                 
 dense_28 (Dense)            (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
lr_schedule = tf.keras.callbacks.LearningRateScheduler(scheduler, verbose=0)

In [ ]:
model.fit(x=x_train,
          y=y_train,
          epochs=5,
          validation_data=(x_test, y_test),
          batch_size = 10000,
          callbacks=[lr_schedule])

Epoch 1/5
6/6 [==============================] - 3s 419ms/step - loss: 6269.2886 - val_loss: 2499.5623 - lr: 0.0010
Epoch 2/5
6/6 [==============================] - 2s 412ms/step - loss: 2393.6536 - val_loss: 1165.3063 - lr: 0.0010
Epoch 3/5
6/6 [==============================] - 2s 405ms/step - loss: 1735.0145 - val_loss: 918.4071 - lr: 1.0000e-04
Epoch 4/5
6/6 [==============================] - 2s 400ms/step - loss: 1533.6758 - val_loss: 882.3265 - lr: 1.0000e-05
Epoch 5/5
6/6 [==============================] - 2s 407ms/step - loss: 1509.8970 - val_loss: 878.1278 - lr: 1.0000e-06
